# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [47]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [48]:
!pip install "dask[dataframe]"

import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [63]:
import os
import sys
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
print(parquet_files) # Checking



['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2009\\part.0.parque

In [56]:
print(dd_px.columns) # Checking

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [66]:
# Write your code below.

dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Adj_Close_lag_1 = x['Adj Close'].shift(1)
                    )
)

dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

dd_range = dd_rets.assign(
    hi_lo_range = dd_rets["High"] - dd_rets["Low"]
)



C:\Users\faisa\AppData\Local\Temp\ipykernel_38676\700305901.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


In [67]:
dd_rets # checking

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns
npartitions=90,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64
ALDX,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
ZS,...,...,...,...,...,...,...,...,...,...,...,...
ZS,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

dd_rets.compute()

dd_moving_avg = dd_rets.groupby('ticker', group_keys=False).apply(
    lambda x: x.sort_values('Date').assign(
        moving_avg_returns_10d = x['Returns'].rolling(window=10).mean()
    )
    
)



C:\Users\faisa\AppData\Local\Temp\ipykernel_38676\2030090418.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_moving_avg = dd_rets.groupby('ticker', group_keys=False).apply(


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It is not necessary to convert to pandas however it make things faster as it takes less memory.
However, if we are working with large data set that takes huge memory then Dask is the option as it works on big data, parallel


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.